# Advent of Code - 2025 - Day 10 - Problem 2

https://adventofcode.com/2025/day/10

## Load Source Data

Load source data into `DATA`.

In [1]:
# Read the input file containing turn instructions
with open("data/day10.txt") as f:
    DATA = [line.strip() for line in f]

# DATA

## Define Machine class

Defines the state of a machine.

We can determine the number of button presses required to produce the specified joltages by solving a series
of linear equations. Example:

```
buttons: [[3], [1, 3], [2], [2, 3], [0, 2], [0, 1]]
valid_joltages: [3, 5, 4, 7]

0P0 + 0P1 + 0P2 + 0P3 + 1P4 + 1P5 = 3
0P0 + 1P1 + 0P2 + 0P3 + 0P4 + 1P5 = 5
0P0 + 0P1 + 1P2 + 1P3 + 1P4 + 0P5 = 4
1P0 + 1P1 + 0P2 + 1P3 + 0P4 + 0P5 = 7
```

In [ ]:
from constraint.problem import Problem
from constraint.constraints import ExactSumConstraint
import re


class Machine:

    def __init__(self, definition: str):

        valid_lights, buttons, valid_joltages = Machine.parse_definition(definition)

        self.valid_lights = valid_lights
        self.buttons = buttons
        self.valid_joltages = valid_joltages

    @staticmethod
    def create_patterns() -> tuple[str, str]:

        ws = r"\s+"
        light_pattern = r"\[([.#]+)\]"
        button_pattern = r"\(([0-9,]+)\)"
        joltage_pattern = r"\{([0-9,]+)\}"

        # Regular expression pattern for the definition string
        line_pattern = (
            f"{light_pattern}((?:{ws}{button_pattern})+){ws}{joltage_pattern}"
        )

        # Regular expression for individual button definitions
        button_pattern = f"(?:{ws}({button_pattern}))"

        return (line_pattern, button_pattern)

    @staticmethod
    def parse_definition(
        definition: str,
    ) -> tuple[list[bool], list[list[int]], list[int]]:

        # Parse the overall definition string. Note that there are two results returned for the buttons due to the nested grouping constructs: the entire
        # button string as well as the last matched button.
        #
        definition_match = re.match(Machine.line_pattern, definition)
        assert definition_match
        buttons = definition_match.group(2)
        button_matches = re.findall(Machine.button_pattern, buttons)

        # Parse and convert the separate machine attributes.
        #
        valid_lights: list[bool] = [light == "#" for light in definition_match.group(1)]
        buttons = [
            list(map(int, button_match[1].split(",")))
            for button_match in button_matches
        ]
        joltages = list(map(int, definition_match.group(4).split(",")))

        return (valid_lights, buttons, joltages)
    
    def get_coeffcients(self) -> list[list[int]]:
    
        # Create a series of linear equations that express joltages in terms of the corresponding buttons.
        result = [[0 for _ in range(len(self.buttons))] for _ in range(len(self.valid_joltages))]
        for idx_button, button in enumerate(self.buttons):
            # print(f"button = {idx_button} - {button}")
            for idx_joltage in button:
                # print(f"  joltagle = {idx_joltage}")
                result[idx_joltage][idx_button] = 1

        return result
    
   def get_coeffcients2(self) -> list[list[int]]:
    
        result = [[0 for _ in range(len(self.buttons)+1)] for _ in range(len(self.valid_joltages))]
        for idx_button, button in enumerate(self.buttons):
            # print(f"button = {idx_button} - {button}")
            for idx_joltage in button:
                # print(f"  joltagle = {idx_joltage}")
                result[idx_joltage][idx_button] = 1
        for idx_joltage, joltage in enumerate(self.valid_joltages):
            result[idx_joltage][len(self.buttons)] = joltage

        return result
        
    def get_solutions(self):

        c = self.get_coeffcients()
        j = self.valid_joltages

        print(j, c)

        max_joltage = max(j)

        problem = Problem()
        for idx_button, button in enumerate(self.buttons):
            max_presses = min(self.valid_joltages[idx_joltage]for idx_joltage in button)
            problem.addVariable(idx_button, range(max_presses + 1))
        # problem.addVariables(range(len(self.buttons)), range(max_joltage + 1))
        for idx_joltage, joltage in enumerate(self.valid_joltages):
            problem.addConstraint(ExactSumConstraint(joltage, c[idx_joltage]))

        solutions = problem.getSolutions()

        print(f"{len(solutions)} solutions found")

        return solutions
    
    def get_min_button_presses(self):
        solutions = self.get_solutions()
        min_button_presses = min(sum(value for value in solution.values()) for solution in solutions)
        return min_button_presses

    line_pattern, button_pattern = create_patterns()

## Solve All Machines

Determine the minimum number of button presses required by all machines.


In [3]:

total = 0

for idx_joltage, specification in enumerate(DATA):
    print(f"{idx_joltage}")
    m = Machine(specification)
    total += m.get_min_button_presses()
    print(f"total = {total}")

total

0
[51, 38, 12, 25, 9, 52, 42, 42, 58, 49] [[1, 0, 0, 1, 0, 1, 1, 0, 0], [1, 0, 1, 0, 0, 0, 1, 1, 0], [0, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 1, 1, 0, 0, 1, 0, 0, 1], [1, 0, 1, 0, 0, 1, 1, 0, 0], [1, 0, 0, 0, 0, 1, 1, 0, 0], [1, 0, 0, 0, 1, 1, 0, 0, 1], [1, 0, 0, 0, 1, 1, 1, 0, 0]]
1 solutions found
total = 88
1
[65, 52, 55, 5, 53, 42, 49] [[1, 1, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 1, 1, 1, 0], [0, 0, 1, 1, 0, 0, 0, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 1, 0, 0, 0], [0, 1, 1, 1, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 0, 1, 1]]
96 solutions found
total = 178
2
[24, 42, 15, 49, 35, 29] [[1, 0, 0, 0, 1], [1, 1, 0, 0, 1], [0, 0, 0, 1, 1], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]
1 solutions found
total = 242
3
[40, 32, 32, 28, 32, 22, 3] [[0, 1, 0, 1, 1, 1], [0, 0, 0, 1, 1, 1], [0, 0, 0, 1, 1, 1], [1, 0, 1, 1, 0, 0], [0, 0, 1, 1, 1, 1], [0, 1, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0]]
1 solutions found
total = 295
4
[86, 57, 6

KeyboardInterrupt: 